<a href="https://colab.research.google.com/github/mmovahed/Applied-Scientific-Computing-with-Python/blob/master/ModelParallel_DNN_MPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  from mpi4py import MPI
except ImportError:
  !pip install mpi4py

In [ ]:
from mpi4py import MPI
import torch
import torch.nn as nn
import torch.optim as optim

# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
size

1

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Define a deep neural network with model parallelism
class ModelParallelNN(nn.Module):
    def __init__(self):
        super(ModelParallelNN, self).__init__()
        self.layer1 = nn.Linear(10, 5)
        self.layer2 = nn.Linear(5, 2)

    def forward(self, x):
        x = self.layer1(x)
        # Send intermediate output to next rank
        # if rank < size - 1:
        #     comm.send(x, dest=rank+1)
        x = self.layer2(x)
        return x

# Initialize model and optimizer
model = ModelParallelNN()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Dummy input
x = torch.randn(10, 10)

# Training loop
for epoch in range(10):
    optimizer.zero_grad()
    output = model(x)
    loss = output.sum()
    loss.backward()
    optimizer.step()
    print(f'Loss: {loss}')
    # Update weights across ranks
    for param in model.parameters():
        if rank > 0:
            recv_buffer = torch.zeros_like(param.data)
            comm.Recv(recv_buffer, source=rank-1)
            param.data += recv_buffer
        if rank < size - 1:
            send_buffer = param.grad.clone()
            comm.Send(send_buffer, dest=rank+1)

# Finalize MPI
comm.Barrier()
if rank == 0:
    print("Training completed!")

Loss: -6.3352885246276855
Loss: -10.414310455322266
Loss: -14.996566772460938
Loss: -20.45527458190918
Loss: -27.27429962158203
Loss: -36.11680603027344
Loss: -47.91712188720703
Loss: -64.00873565673828
Loss: -86.30602264404297
Loss: -117.56443786621094
Training completed!


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!cp gdrive/My\ Drive/path/to/my/file.py .

cp: cannot stat 'gdrive/My Drive/path/to/my/file.py': No such file or directory


In [ ]:
!mpiexec -np 4 python file.py

--------------------------------------------------------------------------
mpiexec has detected an attempt to run as root.

Running as root is *strongly* discouraged as any mistake (e.g., in
defining TMPDIR) or bug can result in catastrophic damage to the OS
file system, leaving your system in an unusable state.

We strongly suggest that you run mpiexec as a non-root user.

You can override this protection by adding the --allow-run-as-root option
to the cmd line or by setting two environment variables in the following way:
the variable OMPI_ALLOW_RUN_AS_ROOT=1 to indicate the desire to override this
protection, and OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 to confirm the choice and
add one more layer of certainty that you want to do so.
We reiterate our advice against doing so - please proceed at your own risk.
--------------------------------------------------------------------------
